# Oguz Kirazdiken BDM Final

ID - 311902013

## Question 1

In [1]:
sudo service postgresql start

Starting PostgreSQL 11 database server: main.


In [8]:
psql -U postgres -c "\l"

                              List of databases
   Name    |  Owner   | Encoding | Collate |  Ctype  |   Access privileges   
-----------+----------+----------+---------+---------+-----------------------
 imdb2     | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 postgres  | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 template0 | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
           |          |          |         |         | postgres=CTc/postgres
 template1 | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
           |          |          |         |         | postgres=CTc/postgres
 world     | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
(5 rows)



### About Dataset

Our dataset consists 41540 international match records between 1872 - 2019. There are also home/away team scores, date and place information.

In [9]:
datadir=~/data

mkdir -p $datadir/results

In [10]:
ls $datadir

comtrade_s1  he_sisli  imdb  ncdc  ngrams  results  world-1.0.tar.gz


In [6]:
wget https://github.com/oguzkirazdiken/oguz.kirazdiken/raw/master/results1.csv -P $datadir/results/

--2020-01-04 18:42:03--  https://github.com/oguzkirazdiken/oguz.kirazdiken/raw/master/results1.csv
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/oguzkirazdiken/oguz.kirazdiken/master/results1.csv [following]
--2020-01-04 18:42:04--  https://raw.githubusercontent.com/oguzkirazdiken/oguz.kirazdiken/master/results1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4300030 (4.1M) [text/plain]
Saving to: ‘/home/jovyan/data/results/results1.csv’

results1.csv        100%[===================>]   4.10M  --.-KB/s    in 0.1s    

2020-01-04 18:42:04 (28.2 MB/s) - ‘/home/jovyan/data/results/results1.csv

In [11]:
ls $datadir/results

results1.csv


In [12]:
createdb -U postgres results

In [13]:
psql -U postgres -c "\l"

                              List of databases
   Name    |  Owner   | Encoding | Collate |  Ctype  |   Access privileges   
-----------+----------+----------+---------+---------+-----------------------
 imdb2     | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 postgres  | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 results   | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 template0 | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
           |          |          |         |         | postgres=CTc/postgres
 template1 | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
           |          |          |         |         | postgres=CTc/postgres
 world     | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
(6 rows)



In [14]:
%load_ext sql

In [15]:
%sql postgres://postgres@localhost/results

'Connected: postgres@results'

In [16]:
%%sql
CREATE TABLE results_table
(
    date text,
    home_team text,
    away_team text,
    home_score integer,
    away_score integer,
    tournament text,
    city text,
    country text,
    neutral text,
    Winner text,
    Loser text,
    Year integer,
    Month integer,
    Day integer,
    total_goals integer  
);

 * postgres://postgres@localhost/results
Done.


[]

In [19]:
psql -U postgres -d results -c "\dt+"

                         List of relations
 Schema |     Name      | Type  |  Owner   |  Size   | Description 
--------+---------------+-------+----------+---------+-------------
 public | results_table | table | postgres | 5920 kB | 
(1 row)



In [18]:
%%sql
COPY results_table FROM '/home/jovyan/data/results/results1.csv' DELIMITER ',' CSV HEADER;

 * postgres://postgres@localhost/results
41540 rows affected.


[]

In [20]:
%%sql
SELECT *
FROM results_table
LIMIT 5;

 * postgres://postgres@localhost/results
5 rows affected.


date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner,loser,year,month,day,total_goals
1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,tie,tie,1872,11,30,0
1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England,Scotland,1873,3,8,6
1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,Scotland,England,1874,3,7,3
1875-03-06,England,Scotland,2,2,Friendly,London,England,False,tie,tie,1875,3,6,4
1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,Scotland,England,1876,3,4,3


### Part 1

The most winning 4 teams and their total wins that they get from the matches with each other.

In [21]:
%%sql

SELECT winner,count(*) as total_wins
FROM results_table
GROUP BY winner
HAVING NOT winner ~* 'tie'
ORDER BY count(*) DESC
LIMIT 4

;

 * postgres://postgres@localhost/results
4 rows affected.


winner,total_wins
Brazil,625
England,572
Germany,555
Argentina,526


In [16]:
%%sql

SELECT winner,count(*) as total_wins_from_eo
FROM results_table
WHERE home_team IN (
        SELECT winner
        FROM results_table
        GROUP BY winner
        HAVING NOT winner ~* 'tie'
        ORDER BY count(*) DESC
        LIMIT 4 ) 
AND away_team IN (
        SELECT winner
        FROM results_table
        GROUP BY winner
        HAVING NOT winner ~* 'tie'
        ORDER BY count(*) DESC
        LIMIT 4 )
GROUP BY winner
HAVING NOT winner ~* 'tie'
ORDER BY count(*) DESC
;

 * postgres://postgres@localhost/results
4 rows affected.


winner,total_wins_from_eo
Brazil,67
Argentina,50
Germany,25
England,23


When we check the total wins from each other, rankings are changing

### Part 2

The most scoring teams in Fifa World Cup matches.

In [17]:
%%sql
WITH away_table AS(
SELECT away_team,sum(away_score) as scores_as_awayteam
FROM results_table
WHERE tournament ~* 'fifa world cup'
GROUP BY away_team
),
home_table AS(
SELECT home_team,sum(home_score) as scores_as_hometeam
FROM results_table
WHERE tournament ~* 'fifa world cup'
GROUP BY home_team
)
SELECT ht.home_team as team,(scores_as_hometeam + scores_as_awayteam) as total_goals
FROM home_table ht
    INNER JOIN away_table at ON ht.home_team = at.away_team
ORDER BY (scores_as_hometeam + scores_as_awayteam) DESC
LIMIT 10
;




 * postgres://postgres@localhost/results
10 rows affected.


team,total_goals
Germany,518
Brazil,455
Mexico,452
Netherlands,382
Australia,378
Spain,375
Argentina,371
England,366
Italy,349
Sweden,347


The most scorer team in fifa world cup matches is Germany !

## Question 2

### About Data

We have New York Times Hardcover Fictions best sellers data from June 7,2008 to July 22,2018

Collected data includes the book title, author, the date of the best seller list, the published date of the list, the book description, the rank (this week and last week), the publisher, number of weeks on the list, and the price.

In [1]:
datadir=~/data

mkdir -p $datadir/bestseller

In [2]:
ls $datadir

bestseller  comtrade_s1  he_sisli  imdb  ncdc  ngrams  world-1.0.tar.gz


In [3]:
wget https://github.com/oguzkirazdiken/oguz.kirazdiken/raw/master/nyt2.json -P $datadir/bestseller/

--2020-01-01 20:16:26--  https://github.com/oguzkirazdiken/oguz.kirazdiken/raw/master/nyt2.json
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/oguzkirazdiken/oguz.kirazdiken/master/nyt2.json [following]
--2020-01-01 20:16:27--  https://raw.githubusercontent.com/oguzkirazdiken/oguz.kirazdiken/master/nyt2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.128.133, 151.101.192.133, 151.101.0.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.128.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6046096 (5.8M) [text/plain]
Saving to: ‘/home/jovyan/data/bestseller/nyt2.json’

nyt2.json           100%[===================>]   5.77M  31.7MB/s    in 0.2s    

2020-01-01 20:16:28 (31.7 MB/s) - ‘/home/jovyan/data/bestseller/nyt2.json’ saved [6

In [4]:
ls $datadir/bestseller

nyt2.json


### Stephen Kings Bestseller Books

Our aim to find Stephen Kings New York Times bestseller books and publisher information

In [5]:
jq -r '. | select(.author == "Stephen King") | .author + "\t" + .title + "\t" + .publisher  ' \
$datadir/bestseller/nyt2.json | \
column -s $'\t' -t | uniq

Stephen King  JUST AFTER SUNSET             Scribner
Stephen King  UNDER THE DOME, PART 1        Scribner
Stephen King  BLOCKADE BILLY                Simon & Schuster
Stephen King  BLOCKADE BILLY                Scribner
Stephen King  FULL DARK, NO STARS           Scribner
Stephen King  11/22/63                      Scribner
Stephen King  THE WIND THROUGH THE KEYHOLE  Scribner
Stephen King  11/22/63                      Scribner
Stephen King  THE WIND THROUGH THE KEYHOLE  Scribner
Stephen King  DOCTOR SLEEP                  Scribner
Stephen King  MR. MERCEDES                  Scribner
Stephen King  REVIVAL                       Scribner
Stephen King  FINDERS KEEPERS               Scribner
Stephen King  THE BAZAAR OF BAD DREAMS      Scribner
Stephen King  END OF WATCH                  Scribner
Stephen King  THE OUTSIDER                  Scribner


## Question 3

Our website Proballers gives basketball stat information about all players in NBA history. Our aim is to find some legendary players average points and rebounds.

In [1]:
datadir=~/data

mkdir -p $datadir/nba

In [2]:
wget --html-extension https://www.proballers.com/basketball/player/2765/michael-jordan -P $datadir/nba/

wget --html-extension https://www.proballers.com/basketball/player/2008/allen-iverson -P $datadir/nba/

wget --html-extension https://www.proballers.com/basketball/player/5002/kareem-abdul-jabbar -P $datadir/nba/

wget --html-extension https://www.proballers.com/basketball/player/2210/karl-malone -P $datadir/nba/

wget --html-extension https://www.proballers.com/basketball/player/2227/kobe-bryant -P $datadir/nba/

wget --html-extension https://www.proballers.com/basketball/player/2327/shaquille-o-neal -P $datadir/nba/

wget --html-extension https://www.proballers.com/basketball/player/2712/wilt-chamberlain -P $datadir/nba/

wget --html-extension https://www.proballers.com/basketball/player/17825/lebron-james -P $datadir/nba/


--2020-01-04 10:19:55--  https://www.proballers.com/basketball/player/2765/michael-jordan
Resolving www.proballers.com (www.proballers.com)... 154.50.0.219
Connecting to www.proballers.com (www.proballers.com)|154.50.0.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/home/jovyan/data/nba/michael-jordan.html’

michael-jordan.html     [ <=>                ]  93.72K  --.-KB/s    in 0.08s   

2020-01-04 10:19:55 (1.14 MB/s) - ‘/home/jovyan/data/nba/michael-jordan.html’ saved [95972]

--2020-01-04 10:19:55--  https://www.proballers.com/basketball/player/2008/allen-iverson
Resolving www.proballers.com (www.proballers.com)... 154.50.0.219
Connecting to www.proballers.com (www.proballers.com)|154.50.0.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/home/jovyan/data/nba/allen-iverson.html’

allen-iverson.html      [ <=>                ]  93.38K  --.-KB/s    in 0.0

In [3]:
ls $datadir/nba/

allen-iverson.html        kobe-bryant.html     shaquille-o-neal.html
kareem-abdul-jabbar.html  lebron-james.html    wilt-chamberlain.html
karl-malone.html          michael-jordan.html


In [15]:
legends=$(for i in $datadir/nba/*; do \
xidel --input-format html --output-format adhoc -e \
"/html/body/main/div/div[3]/div[1]/div/div/div[1]/div/div[2]/h3" \
$i; done)

**** Retrieving: /home/jovyan/data/nba/allen-iverson.html ****
**** Processing: /home/jovyan/data/nba/allen-iverson.html ****
**** Retrieving: /home/jovyan/data/nba/kareem-abdul-jabbar.html ****
**** Processing: /home/jovyan/data/nba/kareem-abdul-jabbar.html ****
**** Retrieving: /home/jovyan/data/nba/karl-malone.html ****
**** Processing: /home/jovyan/data/nba/karl-malone.html ****
**** Retrieving: /home/jovyan/data/nba/kobe-bryant.html ****
**** Processing: /home/jovyan/data/nba/kobe-bryant.html ****
**** Retrieving: /home/jovyan/data/nba/lebron-james.html ****
**** Processing: /home/jovyan/data/nba/lebron-james.html ****
**** Retrieving: /home/jovyan/data/nba/michael-jordan.html ****
**** Processing: /home/jovyan/data/nba/michael-jordan.html ****
**** Retrieving: /home/jovyan/data/nba/shaquille-o-neal.html ****
**** Processing: /home/jovyan/data/nba/shaquille-o-neal.html ****
**** Retrieving: /home/jovyan/data/nba/wilt-chamberlain.html ****
**** Processing: /home/jovyan/data/nba/wil

Our legendary players' list can be seen below

In [16]:
echo "$legends"

Allen Iverson
Kareem Abdul-Jabbar
Karl Malone
Kobe Bryant
Lebron James
Michael Jordan
Shaquille O'neal
Wilt Chamberlain


In [13]:
average_scores=$(for i in $datadir/nba/*; do \
xidel --input-format html --output-format adhoc -e \
"/html/body/main/div/div[3]/div[1]/div/div/div[2]/div/ul/li[1]/p[2]" \
$i; done)

**** Retrieving: /home/jovyan/data/nba/allen-iverson.html ****
**** Processing: /home/jovyan/data/nba/allen-iverson.html ****
**** Retrieving: /home/jovyan/data/nba/kareem-abdul-jabbar.html ****
**** Processing: /home/jovyan/data/nba/kareem-abdul-jabbar.html ****
**** Retrieving: /home/jovyan/data/nba/karl-malone.html ****
**** Processing: /home/jovyan/data/nba/karl-malone.html ****
**** Retrieving: /home/jovyan/data/nba/kobe-bryant.html ****
**** Processing: /home/jovyan/data/nba/kobe-bryant.html ****
**** Retrieving: /home/jovyan/data/nba/lebron-james.html ****
**** Processing: /home/jovyan/data/nba/lebron-james.html ****
**** Retrieving: /home/jovyan/data/nba/michael-jordan.html ****
**** Processing: /home/jovyan/data/nba/michael-jordan.html ****
**** Retrieving: /home/jovyan/data/nba/shaquille-o-neal.html ****
**** Processing: /home/jovyan/data/nba/shaquille-o-neal.html ****
**** Retrieving: /home/jovyan/data/nba/wilt-chamberlain.html ****
**** Processing: /home/jovyan/data/nba/wil

Here we can see these legendary players average scores per match with same order.

In [18]:
echo "$average_scores"

14.3
10.1
13.2
17.6
25.1
20.0
9.2
13.2


In [19]:
average_rebounds=$(for i in $datadir/nba/*; do \
xidel --input-format html --output-format adhoc -e \
"/html/body/main/div/div[3]/div[1]/div/div/div[2]/div/ul/li[2]/p[2]" \
$i; done)

**** Retrieving: /home/jovyan/data/nba/allen-iverson.html ****
**** Processing: /home/jovyan/data/nba/allen-iverson.html ****
**** Retrieving: /home/jovyan/data/nba/kareem-abdul-jabbar.html ****
**** Processing: /home/jovyan/data/nba/kareem-abdul-jabbar.html ****
**** Retrieving: /home/jovyan/data/nba/karl-malone.html ****
**** Processing: /home/jovyan/data/nba/karl-malone.html ****
**** Retrieving: /home/jovyan/data/nba/kobe-bryant.html ****
**** Processing: /home/jovyan/data/nba/kobe-bryant.html ****
**** Retrieving: /home/jovyan/data/nba/lebron-james.html ****
**** Processing: /home/jovyan/data/nba/lebron-james.html ****
**** Retrieving: /home/jovyan/data/nba/michael-jordan.html ****
**** Processing: /home/jovyan/data/nba/michael-jordan.html ****
**** Retrieving: /home/jovyan/data/nba/shaquille-o-neal.html ****
**** Processing: /home/jovyan/data/nba/shaquille-o-neal.html ****
**** Retrieving: /home/jovyan/data/nba/wilt-chamberlain.html ****
**** Processing: /home/jovyan/data/nba/wil

And also, we can see here these players average rebounds per match with same order

In [20]:
echo "$average_rebounds"

2.7
4.5
8.7
3.7
7.6
6.1
4.8
18.6


## Question 5

### About Dataset

Our dataset contains all information about Shakespeare's plays like;
    
- Which characters are involved in a play and character names
    
- How many times a character speak in a play.
    
- Paragraph information in each play and their chapter,section
    
- How many times a word occured in all plays
    

In [1]:
sudo service ssh stop
sudo service ssh start

Stopping OpenBSD Secure Shell server: sshd.
Starting OpenBSD Secure Shell server: sshd.


In [2]:
start-dfs.sh 2> /dev/null

Starting namenodes on [localhost]
localhost: starting namenode, logging to /opt/hadoop-2.9.2/logs/hadoop-jovyan-namenode-jupyter-serhatcevikel-2dbdm-5f2019-2dj11sg2ih.out
localhost: starting datanode, logging to /opt/hadoop-2.9.2/logs/hadoop-jovyan-datanode-jupyter-serhatcevikel-2dbdm-5f2019-2dj11sg2ih.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /opt/hadoop-2.9.2/logs/hadoop-jovyan-secondarynamenode-jupyter-serhatcevikel-2dbdm-5f2019-2dj11sg2ih.out


In [3]:
start-yarn.sh 2> /dev/null

starting yarn daemons
starting resourcemanager, logging to /opt/hadoop-2.9.2/logs/yarn-root-resourcemanager-jupyter-serhatcevikel-2dbdm-5f2019-2dj11sg2ih.out
localhost: starting nodemanager, logging to /opt/hadoop-2.9.2/logs/yarn-jovyan-nodemanager-jupyter-serhatcevikel-2dbdm-5f2019-2dj11sg2ih.out


In [4]:
mr-jobhistory-daemon.sh --config $HADOOP_CONF_DIR start historyserver 2> /dev/null

starting historyserver, logging to /opt/hadoop-2.9.2/logs/mapred-root-historyserver-jupyter-serhatcevikel-2dbdm-5f2019-2dj11sg2ih.out


In [5]:
hdfs dfsadmin -safemode leave 2> /dev/null

Safe mode is OFF


In [6]:
sudo service postgresql start

Starting PostgreSQL 11 database server: main.


In [7]:
datadir=~/data

mkdir -p $datadir/shakespeare

In [8]:
ls $datadir

comtrade_s1  he_sisli  imdb  ncdc  ngrams  shakespeare  world-1.0.tar.gz


In [9]:
wget https://github.com/catherinedevlin/opensourceshakespeare/raw/master/shakespeare.sql -P $datadir/shakespeare/

--2020-01-05 15:33:04--  https://github.com/catherinedevlin/opensourceshakespeare/raw/master/shakespeare.sql
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/catherinedevlin/opensourceshakespeare/master/shakespeare.sql [following]
--2020-01-05 15:33:05--  https://raw.githubusercontent.com/catherinedevlin/opensourceshakespeare/master/shakespeare.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.64.133, 151.101.0.133, 151.101.192.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15049064 (14M) [text/plain]
Saving to: ‘/home/jovyan/data/shakespeare/shakespeare.sql’

shakespeare.sql     100%[===================>]  14.35M  32.1MB/s    in 0.4s    

2020-01-05 15:33:06 (32.1 MB/s) - ‘/hom

In [29]:
ls $datadir/shakespeare/

shakespeare.sql


In [11]:
createdb -U postgres shakespeare

In [12]:
psql -U postgres -c "\l"

                               List of databases
    Name     |  Owner   | Encoding | Collate |  Ctype  |   Access privileges   
-------------+----------+----------+---------+---------+-----------------------
 imdb2       | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 postgres    | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 shakespeare | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 template0   | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
             |          |          |         |         | postgres=CTc/postgres
 template1   | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
             |          |          |         |         | postgres=CTc/postgres
 world       | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
(6 rows)



In [13]:
psql -U postgres shakespeare  < $datadir/shakespeare/shakespeare.sql

SET
SET
SET
SET
SET
SET
CREATE EXTENSION
COMMENT
SET
SET
SET
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
COPY 945
COPY 1266
COPY 1346
COPY 35465
COPY 28830
COPY 43
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
REVOKE
REVOKE
GRANT
GRANT


In [16]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [17]:
%sql postgres://postgres@localhost/shakespeare

'Connected: postgres@shakespeare'

In [19]:
psql -U postgres -d shakespeare -c "\dt+"

                         List of relations
 Schema |      Name      | Type  |  Owner   |  Size   | Description 
--------+----------------+-------+----------+---------+-------------
 public | chapter        | table | postgres | 96 kB   | 
 public | character      | table | postgres | 136 kB  | 
 public | character_work | table | postgres | 96 kB   | 
 public | paragraph      | table | postgres | 15 MB   | 
 public | wordform       | table | postgres | 1760 kB | 
 public | work           | table | postgres | 16 kB   | 
(6 rows)



In [21]:
%%sql

SELECT *
FROM character_work
LIMIT 5 ;

 * postgres://postgres@localhost/shakespeare
5 rows affected.


charid,workid
1apparition-mac,macbeth
1citizen,romeojuliet
1conspirator,coriolanus
1gentleman-oth,othello
1goth,titus


In [17]:
%%sql

SELECT *
FROM work
LIMIT 5 ;

 * postgres://postgres@localhost/shakespeare
5 rows affected.


workid,title,longtitle,year,genretype,notes,source,totalwords,totalparagraphs
12night,Twelfth Night,"Twelfth Night, Or What You Will",1599,c,None,Moby,19837,1031
allswell,All's Well That Ends Well,All's Well That Ends Well,1602,c,None,Moby,22997,1025
antonycleo,Antony and Cleopatra,Antony and Cleopatra,1606,t,None,Moby,24905,1344
asyoulikeit,As You Like It,As You Like It,1599,c,None,Gutenberg,21690,872
comedyerrors,Comedy of Errors,The Comedy of Errors,1589,c,None,Moby,14692,661


In [22]:
hive --service metastore &

[1] 1369


In [23]:
hive -e "create database shakespeare"

2020-01-05 15:35:55: Starting Hive Metastore Server
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.9.2/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.9.2/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Log

: 64

In [24]:
sqoop import \
--connect jdbc:postgresql://localhost:5432/shakespeare \
--username postgres \
--table chapter \
--hive-import --create-hive-table --direct 2> /dev/null

sqoop import \
--connect jdbc:postgresql://localhost:5432/shakespeare \
--username postgres \
--table character \
--hive-import --create-hive-table --direct 2> /dev/null

sqoop import \
--connect jdbc:postgresql://localhost:5432/shakespeare \
--username postgres \
--table character_work \
--hive-import --create-hive-table --direct 2> /dev/null

sqoop import \
--connect jdbc:postgresql://localhost:5432/shakespeare \
--username postgres \
--table paragraph \
--hive-import --create-hive-table --direct 2> /dev/null

sqoop import \
--connect jdbc:postgresql://localhost:5432/shakespeare \
--username postgres \
--table wordform \
--hive-import --create-hive-table --direct 2> /dev/null

sqoop import \
--connect jdbc:postgresql://localhost:5432/shakespeare \
--username postgres \
--table work \
--hive-import --create-hive-table --direct 2> /dev/null


Please set $HBASE_HOME to the root of your HBase installation.
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
Please set $ZOOKEEPER_HOME to the root of your Zookeeper installation.
2020-01-05 15:36:53,076 main ERROR Could not register mbeans java.security.AccessControlException: access denied ("javax.management.MBeanTrustPermission" "register")
	at java.security.AccessControlContext.checkPermission(AccessControlContext.java:472)
	at java.lang.SecurityManager.checkPermission(SecurityManager.java:585)
	at com.sun.jmx.interceptor.DefaultMBeanServerInterceptor.checkMBeanTrustPermission(DefaultMBeanServerInterceptor.java:1848)
	at com.sun.jmx.interceptor.DefaultMBeanServerInterceptor.registerMBean(DefaultMBeanServerInterceptor.java:322)
	at com.sun.jmx.mbeanserver.JmxMBeanServer.registerMBean(JmxMBeanServer.java:522)
	at org.apache.logging.log4j.core.jmx.Server.register(Server.java:380)
	at org.apache.logging.log4j.core.jmx.Server.reregisterMBeansAfterReconfigure(Serve

In [25]:
hive -e 'show tables'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.9.2/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/opt/hive/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
chapter
character
character_work
paragraph
wordform
work
Time taken: 2.2 seconds, Fetched: 6 row(s)


Our aim here is to find total number of characters in shakespeare's each plays and get the all plays which have more than 20 characters

In [26]:
cat > ~/script.pig <<EOF
    work = LOAD 'work' USING org.apache.hive.hcatalog.pig.HCatLoader();
    worked = foreach work generate workid as id_work,title;
    character_work = LOAD 'character_work' USING org.apache.hive.hcatalog.pig.HCatLoader();
    character_worked = foreach character_work generate workid as idwork,charid;
    joined = JOIN worked by id_work, character_worked by idwork;
    selected = foreach joined generate id_work,charid,title;
    grouped = group selected by title;
    charcount = foreach grouped generate group as title,COUNT(selected.charid) as numberofchar;
    ordered = ORDER charcount by numberofchar DESC;
    filtered = filter ordered by numberofchar > 20;
    
    DUMP filtered;
EOF
    

In [28]:
pig -useHCatalog ~/script.pig 2>&1 | grep -Pv "INFO|WARN"

ls: cannot access '/opt/hive/hcatalog/lib/*hbase-storage-handler-*.jar': No such file or directory

HadoopVersion	PigVersion	UserId	StartedAt	FinishedAt	Features
2.9.2	0.17.0	jovyan	2020-01-05 15:47:35	2020-01-05 15:49:59	HASH_JOIN,GROUP_BY,ORDER_BY,FILTER

Success!

Job Stats (time in seconds):
JobId	Maps	Reduces	MaxMapTime	MinMapTime	AvgMapTime	MedianMapTime	MaxReduceTime	MinReduceTime	AvgReduceTime	MedianReducetime	Alias	Feature	Outputs
job_1578238352369_0005	2	1	13	12	12	12	4	4	4	4	character_work,character_worked,joined,selected,work,worked	HASH_JOIN	
job_1578238352369_0006	1	1	5	5	5	5	4	4	4	4	charcount,filtered,grouped	GROUP_BY,COMBINER	
job_1578238352369_0007	1	1	8	8	8	8	7	7	7	7	ordered	SAMPLER	
job_1578238352369_0008	1	1	9	9	9	9	4	4	4	4	ordered	ORDER_BY	hdfs://localhost/tmp/temp1476668196/tmp-1955908881,

Input(s):
Successfully read 43 records from: "work"
Successfully read 1346 records from: "character_work"

Output(s):
Successfully stored 33 records (770 bytes) in: "hdfs://loc